<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true

In [163]:
import pandas as pd
from urllib.request import Request, urlopen
from io import StringIO
import datetime
import time
import random
import math

In [164]:
# dt_from = datetime.date(2000,1,1)
dt_from = datetime.date(2019,1,1)
ut_from = str(int(time.mktime(dt_from.timetuple())))
ut_from

'1546300800'

In [165]:
dt_to = datetime.date.today()
ut_to = str(int(time.mktime(dt_to.timetuple())))
ut_to

'1619740800'

In [166]:
# url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true'
url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=' + ut_from + '&period2=' + ut_to + '&interval=1d&events=history&includeAdjustedClose=true'

In [167]:
headers={'User-Agent': 'Mozilla/5.0'}

In [168]:
req = Request(url, headers=headers)

In [169]:
page = urlopen(req)
# page = urlopen(url)

In [170]:
page = urlopen(url)

In [171]:
content = page.read()

In [172]:
text = content.decode('UTF-8')

In [173]:
print(text[:300])

Date,Open,High,Low,Close,Adj Close,Volume
2019-01-01,1.149425,1.155001,1.146500,1.149306,1.149306,0
2019-01-02,1.146132,1.149700,1.134572,1.146171,1.146171,0
2019-01-03,1.131734,1.140914,1.131734,1.131811,1.131811,0
2019-01-04,1.139095,1.141774,1.134816,1.139108,1.139108,0
2019-01-07,1.141292,1.1474


In [174]:
data = StringIO(text)

In [175]:
df1 = pd.read_csv(data, sep=',', header='infer')

In [176]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume
602,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0
603,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0
604,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0
605,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0
606,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0
607,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0.0
608,2021-04-30,1.212562,1.213298,1.211974,1.211974,1.211974,0.0


In [177]:
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')

In [178]:
df1['DayOfWeek'] = df1['Date'].dt.dayofweek

In [179]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
602,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0,3
603,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0,4
604,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0,0
605,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0,1
606,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0,2
607,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0.0,3
608,2021-04-30,1.212562,1.213298,1.211974,1.211974,1.211974,0.0,4


In [180]:
df2 = df1.drop(['High','Low', 'Adj Close', 'Volume'], axis=1)

In [181]:
df2.tail(7)

,Date,Open,Close,DayOfWeek
602,2021-04-22,1.203818,1.203775,3
603,2021-04-23,1.201793,1.201475,4
604,2021-04-26,1.208956,1.208985,0
605,2021-04-27,1.207921,1.208167,1
606,2021-04-28,1.208854,1.209031,2
607,2021-04-29,1.213578,1.213592,3
608,2021-04-30,1.212562,1.211974,4


In [182]:
df2['Close-1'] = df2['Close'].shift(1)
df2['Open-1'] = df2['Open'].shift(1)

In [183]:
df2.tail(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1
602,2021-04-22,1.203818,1.203775,3,1.203239,1.203254
603,2021-04-23,1.201793,1.201475,4,1.203775,1.203818
604,2021-04-26,1.208956,1.208985,0,1.201475,1.201793
605,2021-04-27,1.207921,1.208167,1,1.208985,1.208956
606,2021-04-28,1.208854,1.209031,2,1.208167,1.207921
607,2021-04-29,1.213578,1.213592,3,1.209031,1.208854
608,2021-04-30,1.212562,1.211974,4,1.213592,1.213578


In [184]:
df2['%Close'] = (df2['Close'] - df2['Close-1']) / df2['Close-1'] * 100
df2['%Open'] = (df2['Open'] - df2['Open-1']) / df2['Open-1'] * 100

In [185]:
df2.head(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open
0,2019-01-01,1.149425,1.149306,1,NaN,NaN,NaN,NaN
1,2019-01-02,1.146132,1.146171,2,1.149306,1.149425,-0.272773,-0.286491
2,2019-01-03,1.131734,1.131811,3,1.146171,1.146132,-1.252867,-1.256225
3,2019-01-04,1.139095,1.139108,4,1.131811,1.131734,0.644719,0.650418
4,2019-01-07,1.141292,1.141044,0,1.139108,1.139095,0.169958,0.192872
5,2019-01-08,1.147934,1.147974,1,1.141044,1.141292,0.607339,0.581972
6,2019-01-09,1.145725,1.145607,2,1.147974,1.147934,-0.206189,-0.192433


In [186]:
def take_decision(index):
  row = df2.loc[index]
  return random.choice([-row['Open'], +row['Open']])

In [187]:
def is_guessed(index):
  row = df2.loc[index]
  if (index < 1):
    return True
  return (math.copysign(1, row['%Open']) == math.copysign(1, row['decision']))

In [188]:
def make_deal(index):
  k = df2.loc[index, 'k'] 
  decision = df2.loc[index, 'decision'] 
  return decision * (2 ** k)

In [189]:
def trade():
  for index, row in df2.iterrows():
    df2.loc[index, 'decision'] = take_decision(index)
    if (is_guessed(index)):
      k = 0
    else:
      k += 1
    df2.loc[index, 'k'] = k
    deal = make_deal(index)
    df2.loc[index, 'deal'] = deal

In [190]:
trade()
df2

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal
0,2019-01-01,1.149425,1.149306,1,NaN,NaN,NaN,NaN,1.149425,0.0,1.149425
1,2019-01-02,1.146132,1.146171,2,1.149306,1.149425,-0.272773,-0.286491,1.146132,1.0,2.292264
2,2019-01-03,1.131734,1.131811,3,1.146171,1.146132,-1.252867,-1.256225,1.131734,2.0,4.526936
3,2019-01-04,1.139095,1.139108,4,1.131811,1.131734,0.644719,0.650418,-1.139095,3.0,-9.112760
4,2019-01-07,1.141292,1.141044,0,1.139108,1.139095,0.169958,0.192872,1.141292,0.0,1.141292
...,...,...,...,...,...,...,...,...,...,...,...
604,2021-04-26,1.208956,1.208985,0,1.201475,1.201793,0.625065,0.596026,-1.208956,3.0,-9.671648
605,2021-04-27,1.207921,1.208167,1,1.208985,1.208956,-0.067660,-0.085611,-1.207921,0.0,-1.207921
606,2021-04-28,1.208854,1.209031,2,1.208167,1.207921,0.071513,0.077240,1.208854,0.0,1.208854
607,2021-04-29,1.213578,1.213592,3,1.209031,1.208854,0.377244,0.390783,-1.213578,1.0,-2.427156


In [191]:
df2.describe()

,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal
count,588.000000,588.000000,609.000000,587.000000,587.000000,584.000000,584.000000,588.000000,609.00000,588.000000
mean,1.142495,1.142544,2.003284,1.142426,1.142376,0.011039,0.011126,0.016635,1.08046,0.153967
std,0.041087,0.041088,1.414210,0.041023,0.041020,0.401315,0.398889,1.144085,1.41599,14.799617
min,1.065848,1.065735,0.000000,1.065735,1.065848,-2.775155,-2.603845,-1.233776,0.00000,-157.923328
25%,1.111454,1.111565,1.000000,1.111538,1.111451,-0.231582,-0.233792,-1.129356,0.00000,-2.205676
50%,1.128859,1.128796,2.000000,1.128796,1.128796,0.018168,0.014842,1.080655,1.00000,1.084770
75%,1.181384,1.181513,3.000000,1.181404,1.181356,0.243664,0.246487,1.128537,2.00000,2.191916
max,1.233776,1.234111,4.000000,1.234111,1.233776,1.457266,1.456985,1.226196,7.00000,140.381696
